In [1]:
import glob
import os
import shutil
import time
import numpy as np
from PIL import Image
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import cropLargestRecT, getMeanTiff_randomsampling
from utils_io import get_imaging_files, get_rotary_center
from scanimagetiffio import SITiffIO

In [18]:
#read all the imaging files in the folder
datafolder = "/home/zilong/Desktop/2D2P/Data/183_27072023/"
namelist = ["00003", "00004", "00005", "00006", "00007", "00008", "00009", "00012", "00013", "00014", "00018"]

allfiles = get_imaging_files(datafolder, namelist, readVRlogs=False)
#print all the pairs
for pair in allfiles:
    print(pair)

['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00003.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_161337.00003.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00004.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_162227.00004.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00005.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00006.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_164024.00006.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00007.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_164812.00007.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00008.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_165553.00008.txt']
['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00009.tif', '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata

In [3]:
# read the rotation center from the circlecenter txt file
circlecenterfilename = datafolder+"DP_exp/circlecenter.txt"
rotCenter = get_rotary_center(circlecenterfilename)
print("Rotation center is at ({}, {})".format(rotCenter[0], rotCenter[1]))

Rotation center is at (255.0, 242.0)


In [16]:
#get the median value of the mean frame for each tiff file
all_medians = []
for i, (tiff, _)  in enumerate(allfiles):
    print('Get the median value of tiff file: ', tiff)
    S = SITiffIO()
    S.open_tiff_file(tiff, "r")
    meanFrame = getMeanTiff_randomsampling(S, frac=0.01)
    medianVal = np.median(meanFrame)
    #store the median value
    all_medians.append(medianVal)

Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00003.tif
Randomly select 83 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00004.tif
Randomly select 90 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00005.tif
Randomly select 90 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00006.tif
Randomly select 90 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00007.tif
Randomly select 90 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00008.tif
Randomly select 90 frames to average over...
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00009.tif
Randomly select 90 frames to average 

In [21]:
#get the histgram offset 
histoffset = [all_medians[0]-all_medians[i] for i in range(len(all_medians))]

#create a folder named UnrotTiff under the parent folder
UnrotTiffFolder = datafolder + "UnrotTiff_Matched/"
#if the folder exists, remove it, and then create a new one
if os.path.exists(UnrotTiffFolder):
    shutil.rmtree(UnrotTiffFolder)
os.makedirs(UnrotTiffFolder)

#for each pair of tiff file and RElog file, unrotate the tiff file, match the histogram, and save it into a folder named UnrotTiff
for i, (tiff_file, RElog_file) in enumerate(allfiles):
    
    #time
    t = time.time()

    #geterante the unrotated tiff file name
    unrotated_tiff_file = UnrotTiffFolder+tiff_file.split("/")[-1].split(".")[0] + "_unrot.tif"
    #print processing file name
    print("Unrotating tiff file: " + tiff_file)

    #get histogram offset
    offset = histoffset[i]
  
    #unrotate the tiff file and save it into the UnrotTiff folder
    S = SITiffIO()
    S.open_tiff_file(tiff_file, "r")
    S.open_tiff_file(unrotated_tiff_file, "w") #for writing
    S.open_rotary_file(RElog_file)
    S.interp_times()  # might take a while...
    
    N = S.get_n_frames()
    Alltheta = S.get_all_theta()

    for i in range(N):
        frame_i = S.get_frame(i+1)
        theta_i = Alltheta[i]
        #unrotate the frame
        unrotatedFrame = Image.fromarray(frame_i).rotate(theta_i, center=rotCenter)
        #crop the largest inner rectangle from the unrotated frame
        croppedFrame = cropLargestRecT(unrotatedFrame, rotCenter)
        #convert the PIL image back to int16
        croppedFrame = np.array(croppedFrame, dtype=np.int16)+offset

        S.write_frame(croppedFrame, i)
    
    #time
    print("Time for unrotating current tiff file: " + str(time.time()-t))

Unrotating tiff file: /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00003.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_161337.00003.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 27032 timestamps in it.
Counted 8355 frames
Unrotating tiff file: /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00004.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_162227.00004.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 27445 timestamps in it.
Counted 9000 frames
Unrotating tiff file: /home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00005.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt
Calculating rotations and times from rotary encoder data...
Finished calculatin

In [1]:
import sys
sys.path.append("..")
from utils_image import UnrotateTiff

datafolder = '/home/zilong/Desktop/2D2P/Data/183_03082023/'
UnrotateTiff(datafolder=datafolder, 
             namelist = ["00004", "00005", "00006", "00007", "00008", "00009", "00010", "00012", "00014", "00016", "00017", "00018"],
             readVRlogs = False)


Unrotate Imaging Tiff files...
Rotation center is at (245.0, 248.0)
Get the median value of tiff file:  /home/zilong/Desktop/2D2P/Data/183_03082023/03082023_00001.tif
Randomly select 20 frames to average over...
